In [9]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
# Define global variables for the analysis

# Define the path to the data
phospho_matrix = 'C:/Users/plutzer/Box/CellBio-MajorLab/Users/Isaac/Experiments/013_NEKs_phospho/Analysis/Combined/Phospho/report.pr_matrix.tsv'
protein_matrix = 'C:/Users/plutzer/Box/CellBio-MajorLab/Users/Isaac/Experiments/013_NEKs_phospho/Analysis/Combined/WC/report.pg_matrix.tsv'

# Database path
database_path = 'C:/Users/plutzer/Box/CellBio-MajorLab/Users/Isaac/Experiments/013_NEKs_phospho/Analysis/Combined/Human_sp_20230119_3AUP00000564.fasta'

# Define an output folder
output_folder_path = 'C:/Users/plutzer/Box/CellBio-MajorLab/Users/Isaac/Experiments/013_NEKs_phospho/Analysis/Combined/Output/'

# Define a pathname or the proximity data
proximity_file_name = '/Users/plutzer/Box/CellBio-MajorLab/Users/Isaac/Experiments/Projects/IDG_pipeline/137_DKs/Annotated_Merged_Output.csv' # desktop path

# Define a dictionary to map column names
columns_mapping = {
    'MA5890_293T_IBP_DMSO_1_Phospho.raw':'DMSO 1',
    'MA5891_293T_IBP_DMSO_2_Phospho.raw':'DMSO 2',
    'MA5892_293T_IBP_DMSO_3_Phospho_20231023164958.raw':'DMSO 3',
    'MA5893_293T_IBP_DMSO_4_Phospho_20231023182154.raw':'DMSO 4',
    'MA5894_293T_IBP_CDDO_1_Phospho_20231023195348.raw':'CDDO 1',
    'MA5895_293T_IBP_CDDO_2_Phospho_20231023212542.raw':'CDDO 2',
    'MA5896_293T_IBP_CDDO_3_Phospho_20231023225736.raw':'CDDO 3',
    'MA5897_293T_IBP_CDDO_4_Phospho_20231024002929.raw':'CDDO 4',
    'MA5898_293T_IBP_dsRed_1_Phospho_20231025225323.raw':'dsRed 1',
    'MA5899_293T_IBP_dsRed_2_Phospho_20231026002524.raw':'dsRed 2',
    'MA5900_293T_IBP_dsRed_3_Phospho_20231026015719.raw':'dsRed 3',
    'MA5901_293T_IBP_dsRed_4_Phospho_20231026032913.raw':'dsRed 4',
    'MA5914_293T_IBP_AKT1_1_Phospho.raw':'AKT1 1',
    'MA5915_293T_IBP_AKT1_2_Phospho.raw':'AKT1 2',
    'MA5916_293T_IBP_AKT1_3_Phospho.raw':'AKT1 3',
    'MA5917_293T_IBP_AKT1_4_Phospho.raw':'AKT1 4',
    'MA5922_293T_IBP_BRSK2_1_Phospho.raw':'BRSK2 1',
    'MA5923_293T_IBP_BRSK2_2_Phospho.raw':'BRSK2 2',
    'MA5924_293T_IBP_BRSK2_3_Phospho.raw':'BRSK2 3',
    'MA5925_293T_IBP_BRSK2_4_Phospho.raw':'BRSK2 4'
}

quant_prefix = 'D:\\MaxQuant_Searches\\Major lab\\Isaac\\013_Phospho_DIA_Kingfisher\\Kinases\\Phospho\\'

quant_cols_order = [
    'DMSO 1',
    'DMSO 2',
    'DMSO 3',
    'DMSO 4',
    'CDDO 1',
    'CDDO 2',
    'CDDO 3',
    'CDDO 4',
    'dsRed 1',
    'dsRed 2',
    'dsRed 3',
    'dsRed 4',
    'AKT1 1',
    'AKT1 2',
    'AKT1 3',
    'AKT1 4',
    'BRSK2 1',
    'BRSK2 2',
    'BRSK2 3',
    'BRSK2 4'
]



In [3]:
# If the output directory doesn't exist, create it
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [4]:
# Code for mixing correction:
def correct_mixing(dataset,columns):
    data = dataset[columns]
    sums = np.sum(data)
    corrected = data/(sums/np.mean(sums))
    return corrected

In [5]:
# Read in the phospho data
phospho = pd.read_csv(phospho_matrix,sep='\t')

# Parse the database file line by line
# Create a dictionary to store the protein sequences
protein_sequences = {}

# Open the database file
with open(database_path,'r') as f:
    # Iterate over the lines in the file
    for line in f:
        # Check if the line is a header line
        if line[0] == '>':
            # If it is, parse the line to get the protein name
            protein_name = line.split('|')[1]
        else:
            # If it isn't a header line, it's part of the sequence
            # Add the sequence to the sequence string
            # Check if the protein name is already in the dictionary
            if protein_name in protein_sequences:
                # If it is, add the sequence to the existing sequence
                protein_sequences[protein_name] += line.strip()
            else:
                # If it isn't, initialize the sequence
                protein_sequences[protein_name] = line.strip()

# Create a function that takes in a protein name and peptide sequence and fin


In [35]:
def find_peptide_start(protein_name, peptide_sequence):
    start_positions = []
    if protein_name in protein_sequences:
        protein_sequence = protein_sequences[protein_name]
        for i in range(len(protein_sequence) - len(peptide_sequence) + 1):
            if protein_sequence[i:i+len(peptide_sequence)] == peptide_sequence:
                start_positions.append(i)
        if len(start_positions) == 0:
            print("Peptide sequence cannot be mapped to the protein sequence.")
        # elif len(start_positions) > 1:
        #     a = 0
        # else:
        #     a = 0
        #     #print("Peptide sequence starts at position:", start_positions[0])
    else:
        print("Protein name not found in the protein sequences dictionary.")
    return start_positions

# Now need a second function that finds the position of the modified residue within the peptide sequence and adds it to the start position
def find_mod_position(mod_sequence, start_positions):
    # Clean up the mod sequence

    # Replace (UniMod:21) with *
    mod_sequence = re.sub(r'\(UniMod:21\)', '*', mod_sequence)

    # Remove all text that is inside of parentheses
    mod_sequence = re.sub(r'\([^()]*\)', '', mod_sequence)

    

    mod_positions = []
    for start in start_positions:
        mod_positions += list((np.array([len(item) for item in mod_sequence.split('*')[:-1]]) + start - 1).astype(str))
    return ';'.join(mod_positions)

# Create a function that creates an index for a row by combining the Protein.ID and modified_position columns with the residue that is modified.
def create_index(id,mod_sequence,mod_position):
    mod_res = mod_sequence.split('(UniMod:21)')[0][-1]
    return id + '_' + mod_res + mod_position



In [40]:
# Split the Protein.Group column of the phospho dataframe to get the first protein id separated by ';'
phospho['first_id'] = phospho['Protein.Group'].str.split(';').str[0]

# Apply the find_peptide_start function to the first_id column and peptide sequence column
phospho['start_position'] = phospho.apply(lambda row: find_peptide_start(row['first_id'], row['Stripped.Sequence']), axis=1)

# Apply the find_mod_position function to the stripped sequence column and start position column
phospho['mod_position'] = phospho.apply(lambda row: find_mod_position(row['Modified.Sequence'], row['start_position']), axis=1)

# Filter the phospho dataframe to only include rows where the mod_position column is not empty and doesn't contain a semicolon
phospho = phospho[phospho['mod_position'] != '']
phospho = phospho[~phospho['mod_position'].str.contains(';')]

# Create a column for the first Gene in the Genes column
phospho['first_gene'] = phospho['Genes'].str.split(';').str[0]

# Create an index using the create_index function
phospho['index'] = phospho.apply(lambda row: create_index(row['first_gene'], row['Modified.Sequence'], row['mod_position']), axis=1)

# Set the index of the phospho dataframe to the index column
phospho.set_index('index',inplace=True)

# Save the phospho dataframe to a csv file
phospho.to_csv(output_folder_path + 'indexed_phospho.csv')

In [32]:
phospho

,Protein.Group,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Proteotypic,Stripped.Sequence,Modified.Sequence,Precursor.Charge,Precursor.Id,...,D:\MaxQuant_Searches\Major lab\Isaac\013_Phospho_DIA_Kingfisher\Kinases\Phospho\MA5915_293T_IBP_AKT1_2_Phospho.raw,D:\MaxQuant_Searches\Major lab\Isaac\013_Phospho_DIA_Kingfisher\Kinases\Phospho\MA5916_293T_IBP_AKT1_3_Phospho.raw,D:\MaxQuant_Searches\Major lab\Isaac\013_Phospho_DIA_Kingfisher\Kinases\Phospho\MA5917_293T_IBP_AKT1_4_Phospho.raw,D:\MaxQuant_Searches\Major lab\Isaac\013_Phospho_DIA_Kingfisher\Kinases\Phospho\MA5922_293T_IBP_BRSK2_1_Phospho.raw,D:\MaxQuant_Searches\Major lab\Isaac\013_Phospho_DIA_Kingfisher\Kinases\Phospho\MA5923_293T_IBP_BRSK2_2_Phospho.raw,D:\MaxQuant_Searches\Major lab\Isaac\013_Phospho_DIA_Kingfisher\Kinases\Phospho\MA5924_293T_IBP_BRSK2_3_Phospho.raw,D:\MaxQuant_Searches\Major lab\Isaac\013_Phospho_DIA_Kingfisher\Kinases\Phospho\MA5925_293T_IBP_BRSK2_4_Phospho.raw,first_id,start_position,mod_position
0,P0CG40,P0CG40,SP9_HUMAN,SP9,Transcription factor Sp9,1,AAAAAAAAAAAAAAAASAGGK,AAAAAAAAAAAAAAAASAGGK,3,AAAAAAAAAAAAAAAASAGGK3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P0CG40,[454],
1,P55011,P55011;G3XAL9,S12A2_HUMAN,SLC12A2,Solute carrier family 12 member 2,1,AAAAAAAAAAAAAAAGAGAGAK,AAAAAAAAAAAAAAAGAGAGAK,3,AAAAAAAAAAAAAAAGAGAGAK3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P55011,[92],
2,P35453,P35453,HXD13_HUMAN,HOXD13,Homeobox protein Hox-D13,1,AAAAAAAAAAAAAAASGFAYPGTSER,AAAAAAAAAAAAAAASGFAYPGTSER,3,AAAAAAAAAAAAAAASGFAYPGTSER3,...,52097.9,NaN,NaN,NaN,NaN,NaN,NaN,P35453,[56],
3,Q8WXD9,Q8WXD9,CSKI1_HUMAN,CASKIN1,Caskin-1,1,AAAAAAAAAAAPPAPPEGASPGDSAR,AAAAAAAAAAAPPAPPEGAS(UniMod:21)PGDSAR,3,AAAAAAAAAAAPPAPPEGAS(UniMod:21)PGDSAR3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q8WXD9,[1344],1363
4,Q8WXD9,Q8WXD9,CSKI1_HUMAN,CASKIN1,Caskin-1,1,AAAAAAAAAAAPPAPPEGASPGDSAR,AAAAAAAAAAAPPAPPEGASPGDS(UniMod:21)AR,3,AAAAAAAAAAAPPAPPEGASPGDS(UniMod:21)AR3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q8WXD9,[1344],1367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26595,Q6P2Q9,Q6P2Q9;I3L0J9,PRP8_HUMAN,PRPF8,Pre-mRNA-processing-splicing factor 8,1,YYVLNALK,YYVLNALK,2,YYVLNALK2,...,NaN,NaN,NaN,NaN,NaN,NaN,131634.0,Q6P2Q9,[452],
26596,P68104,P68104;Q5VTE0;A0A087WV01;A0A087WVQ9;A0A7I2V3H3...,EF1A1_HUMAN,EEF1A1,Elongation factor 1-alpha 1,0,YYVTIIDAPGHR,YYVTIIDAPGHR,2,YYVTIIDAPGHR2,...,5020840.0,2645430.0,1312910.0,1301690.0,1006220.0,1857490.0,3021060.0,P68104,[84],
26597,P68104,P68104;Q5VTE0;A0A087WV01;A0A087WVQ9;A0A7I2V3H3...,EF1A1_HUMAN,EEF1A1,Elongation factor 1-alpha 1,0,YYVTIIDAPGHR,YYVTIIDAPGHR,3,YYVTIIDAPGHR3,...,20940100.0,11192900.0,6437000.0,5778920.0,4527190.0,7658360.0,12969600.0,P68104,[84],
26598,P68104,P68104;Q5VTE0;A0A087WV01;A0A087WVQ9;A0A7I2V3H3...,EF1A1_HUMAN,EEF1A1,Elongation factor 1-alpha 1,0,YYVTIIDAPGHRDFIK,YYVTIIDAPGHRDFIK,3,YYVTIIDAPGHRDFIK3,...,142451.0,42092.6,NaN,138382.0,NaN,75477.8,138054.0,P68104,[84],
